In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=ffe8b2ef4715cf4d2c5f78fad708b5ff3b9648353bbf5f18c42b863f23dc3774
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=85e9f0c8754b23a78d6260d183e3ed9c332d30545026cdb9d09b0559f1a46e70
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.3.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5087 - accuracy: 0.8170 - val_loss: 0.4324 - val_accuracy: 0.8378
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3889 - accuracy: 0.8554 - val_loss: 0.4560 - val_accuracy: 0.8370
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3588 - accuracy: 0.8667 - val_loss: 0.3705 - val_accuracy: 0.8640


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4979 - accuracy: 0.8278 - val_loss: 0.4163 - val_accuracy: 0.8567
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3645 - accuracy: 0.8670 - val_loss: 0.4448 - val_accuracy: 0.8592
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3302 - accuracy: 0.8799 - val_loss: 0.3613 - val_accuracy: 0.8722


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4951 - accuracy: 0.8243 - val_loss: 0.4397 - val_accuracy: 0.8403
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3811 - accuracy: 0.8609 - val_loss: 0.4057 - val_accuracy: 0.8512
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3482 - accuracy: 0.8716 - val_loss: 0.3791 - val_accuracy: 0.8598


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5841 - accuracy: 0.7935 - val_loss: 0.4347 - val_accuracy: 0.8350
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4198 - accuracy: 0.8444 - val_loss: 0.4391 - val_accuracy: 0.8453
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3856 - accuracy: 0.8564 - val_loss: 0.4041 - val_accuracy: 0.8500


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 2.3658 - accuracy: 0.0999 - val_loss: 2.3025 - val_accuracy: 0.1008
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 2.3041 - accuracy: 0.0981 - val_loss: 2.3045 - val_accuracy: 0.1032
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 2.3041 - accuracy: 0.0983 - val_loss: 2.3034 - val_accuracy: 0.0985


INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,661,354
Trainable params: 2,661,354
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=50, validation_split=0.1, initial_epoch=3)

Epoch 4/50
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3032 - accuracy: 0.8887 - val_loss: 0.3617 - val_accuracy: 0.8757
Epoch 5/50
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2805 - accuracy: 0.8979 - val_loss: 0.3725 - val_accuracy: 0.8648
Epoch 6/50
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2637 - accuracy: 0.9047 - val_loss: 0.4169 - val_accuracy: 0.8765
Epoch 7/50
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2505 - accuracy: 0.9091 - val_loss: 0.3929 - val_accuracy: 0.8762
Epoch 8/50
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2278 - accuracy: 0.9177 - val_loss: 0.4690 - val_accuracy: 0.8708
Epoch 9/50
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2273 - accuracy: 0.9184 - val_loss: 0.4600 - val_accuracy: 0.8740
Epoch 10/50
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2233 - accuracy: 0.9216 - val_loss: 0.5132 - val_accuracy